In [1]:
import pandas as pd
import numpy as np
import networkx as nx 

import matplotlib.pyplot as plt
from ast import literal_eval
import time 


from os.path import join
import os

from dotenv import load_dotenv
load_dotenv()  

path = os.environ['PROJECT_PATH']
covalent_key = os.environ['COVALENTHQ_API_KEY']

from dotenv import load_dotenv
load_dotenv()  



/home/user/eisermann/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/user/eisermann/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


True

In [20]:
# load snapshots
df_snapshots = pd.read_csv('../assets/snapshot_selection.csv')
df_snapshots['Date'] = pd.to_datetime(df_snapshots['Date'])

# load tokens - act as ground truth 
# df_tokens = pd.read_csv('../assets/df_final_token_selection_20230813.csv')
df_tokens = pd.read_csv('../assets/archive/df_final_token_selection_20221209.csv')

,Nr.,Block Height,Date
0,1,7962629,2019-06-15
1,2,8155117,2019-07-15
2,3,8354625,2019-08-15
3,4,8553607,2019-09-15
4,5,8745378,2019-10-15
5,6,8938208,2019-11-15
6,7,9110216,2019-12-15
7,8,9285458,2020-01-15
8,9,9487426,2020-02-15
9,10,9676110,2020-03-15


In [13]:
import requests

def convert_blockheight_to_date(snapshot, covalent_key): 
    
    """Takes block_height and returns date.

    Args:
        snapshot (str): block height

    Returns:
        str: date of format YYYY-MM-DD

    Raises:
        None
    """
    
    url = f"https://api.covalenthq.com/v1/eth-mainnet/block_v2/{snapshot}/?key={covalent_key}"

    response = requests.get(url)
    
    print(response)
    date = response.json()['data']['items'][0]['signed_at']
    
    
    
    date_YMD = date.split('T')[0]
    
    return date_YMD

In [37]:
def usd_value_decimals_token(snapshot, contractAddress, covalent_key, date, chainName = 'eth-mainnet', quoteCurrency = 'USD'): 
    
    """Takes blockheight and contract address of token and
    returns contract_decimals and value in usd.

    Args:
        snapshot (str): block height.
        contractAddress (str): token contract address

    Returns:
        dict with keys and value pairs: 
        'contract_address': contractAddress (str)
        'token_price_usd': price_usd_per_token (int)
        'contract_decimals': decimals (int)

    Raises:
        None
    """
    
    # date = convert_blockheight_to_date(snapshot, covalent_key)
    date = date.strftime('%Y-%m-%d') #assumes date time object
    
    url = f"https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/{chainName}/{quoteCurrency}/{contractAddress}/?key={covalent_key}"    
    params = {
    "from": date, # (YYYY-MM-DD)
    "to": date # (YYYY-MM-DD)
    }

    response = requests.get(url, params=params)
    
    # token price 
    
    try: 
        price_usd_per_token = response.json()['data'][0]['prices'][0]['price']
    
    except: 
        price_usd_per_token = np.nan
    
    # decimals
    decimals = response.json()['data'][0]['contract_decimals']
    
    
    return {'snapshot': snapshot,'contract_address': contractAddress, 'token_price_usd': price_usd_per_token, 'contract_decimals': decimals}

In [50]:
data_timestamps = {}

for _, row in df_snapshots.iterrows():
    
    data_timestamp ={}
    for token_address in df_tokens.address:
                
        snapshot = row['Block Height']
        date = row['Date']
    
        results = usd_value_decimals_token(snapshot, token_address, covalent_key, date, chainName = 'eth-mainnet', quoteCurrency = 'USD')
        
        data_timestamp[results['contract_address']] =  results['token_price_usd']
        
        time.sleep(0.5)  # Sleep for 0.2 seconds to limit to 5 requests per second
        
    data_timestamps[results['snapshot']] = data_timestamp
        
        



In [57]:
df_prices = pd.DataFrame(data_timestamps) 
df_prices.to_csv('price_table.csv')


In [56]:
df_prices

,7962629,8155117,8354625,8553607,8745378,8938208,9110216,9285458,9487426,9676110,...,13230157,13422506,13620205,13809597,14009885,14210564,14391029,14589816,14779829,14967365
0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.324745,25.964787,24.079120,15.229941,16.516690,11.446650,8.701862,9.776532,5.453486,4.390782
0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,411.706570,302.326000,303.330600,173.686860,240.104300,172.028240,121.911090,176.219060,92.165130,66.191740
0x5a98fcbea516cf06857215779fd812ca3bef1b32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.173432,4.034254,4.268341,3.003296,2.522548,1.764966,2.670974,3.022245,1.475332,0.618067
0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,728.09875,592.3968,476.78873,471.92816,498.22504,619.5419,493.59546,529.3737,602.0609,213.08313,...,3058.302700,2577.520300,3030.956000,2382.588600,2222.775000,2090.613000,1786.493000,1947.676300,1564.728900,824.640440
0xd533a949740bb3306d119cc777fa900ba034cd52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.029544,2.912244,3.952522,3.736292,5.310106,3.170005,1.952767,2.225204,1.506180,0.765545
0x111111111117dc0aa78b770fa6a738034120c302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.292374,3.254085,4.204173,2.550961,2.388826,1.794362,1.351426,1.519230,0.978955,0.628206
0x1a4b46696b2bb4794eb3d4c26f1c55f9170fa4c5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.704174,1.785794,2.694689,1.830235,2.006870,1.687911,1.098497,1.226379,0.866328,0.443774
0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.781469,15.595557,29.832237,28.911524,45.726086,29.018856,17.403570,28.677980,11.785809,4.091065
0xc00e94cb662c3520282e6f5717214004a7f26888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,442.069150,318.277250,333.669300,191.807530,190.896130,135.915660,104.249300,147.793030,74.924290,35.384674
0x92d6c1e31e14520e676a687f0a93788b716beff5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.533859,22.725151,14.340446,8.585098,7.912137,7.572128,4.496718,4.673142,2.139968,1.284379
